# Gemma 3 9B Fine-tuning on H100

고전문학 사고유도 AI 모델 학습

## 1. 환경 설정

In [ ]:
# 호환되는 버전으로 설치 (PyTorch 2.4.0 + Transformers 4.44.2)
!pip uninstall -y torch torchvision torchaudio transformers peft trl bitsandbytes numpy scipy

!pip install numpy==1.26.4 scipy==1.13.1

!pip install torch==2.4.0 torchvision==0.19.0 torchaudio==2.4.0 --index-url https://download.pytorch.org/whl/cu121

!pip install \
  "transformers==4.44.2" \
  "peft==0.12.0" \
  "trl==0.9.6" \
  "bitsandbytes==0.43.3" \
  "datasets==2.21.0" \
  "accelerate==0.33.0"
  
print("✅ 설치 완료! 이제 Kernel을 재시작하세요.")

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from datasets import load_dataset
from trl import SFTTrainer
import os

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"GPU count: {torch.cuda.device_count()}")
if torch.cuda.is_available():
    print(f"GPU name: {torch.cuda.get_device_name(0)}")

## 1.1 GPU 상세 확인 (중요!)

In [ ]:
# GPU 상세 정보 확인
import torch

print("="*60)
print("GPU 상태 확인")
print("="*60)

print(f"\n✅ CUDA 사용 가능: {torch.cuda.is_available()}")
print(f"✅ GPU 개수: {torch.cuda.device_count()}")

if torch.cuda.is_available():
    for i in range(torch.cuda.device_count()):
        props = torch.cuda.get_device_properties(i)
        print(f"\n🔥 GPU {i}: {torch.cuda.get_device_name(i)}")
        print(f"   메모리: {props.total_memory / 1024**3:.2f} GB")
        print(f"   Compute Capability: {props.major}.{props.minor}")
        print(f"   멀티프로세서: {props.multi_processor_count}")
        
    # 현재 메모리 사용량
    print(f"\n💾 현재 메모리 사용:")
    for i in range(torch.cuda.device_count()):
        allocated = torch.cuda.memory_allocated(i) / 1024**3
        reserved = torch.cuda.memory_reserved(i) / 1024**3
        print(f"   GPU {i}: {allocated:.2f}GB allocated, {reserved:.2f}GB reserved")
else:
    print("⚠️ GPU를 찾을 수 없습니다!")
    
print("\n" + "="*60)

## 2. 데이터 로드

In [ ]:
# Cloud Storage에서 균형잡힌 데이터 다운로드
!gsutil cp gs://knu-team-03-data/classical-literature/gemma/train_balanced.jsonl ./
!gsutil cp gs://knu-team-03-data/classical-literature/gemma/valid_balanced.jsonl ./

print("✅ 균형잡힌 데이터셋 다운로드 완료!")
print("   - 고전문학 50% + AI Hub 우수 샘플 50%")

## 2.1 Hugging Face 로그인 (중요!)

In [ ]:
# Hugging Face 로그인
from huggingface_hub import login

# Hugging Face 토큰
HF_TOKEN = "YOUR_HF_TOKEN_HERE"

login(token=HF_TOKEN)
print("✅ Hugging Face 로그인 완료!")
print("✅ Gemma 모델 접근 권한 확인됨")

In [ ]:
# 균형잡힌 데이터셋 로드
train_dataset = load_dataset('json', data_files='train_balanced.jsonl', split='train')
valid_dataset = load_dataset('json', data_files='valid_balanced.jsonl', split='train')

print(f"✅ 균형잡힌 데이터셋 로드 완료!")
print(f"   Train: {len(train_dataset)}개 (고전 50% + AI Hub 우수 50%)")
print(f"   Valid: {len(valid_dataset)}개")
print(f"\n⚡ 예상 학습 시간: 30-40분 (기존 6시간의 1/10)")
print(f"\nSample: {train_dataset[0]}")

## 3. 모델 및 토크나이저 로드

In [ ]:
model_name = "google/gemma-2-9b-it"

# 4-bit 양자화 설정
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# 모델 로드 (Gemma2 최적화: eager attention)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    attn_implementation="eager"  # Gemma2 권장 설정
)

# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

print("✅ 모델 로드 완료 (eager attention)")

## 4. LoRA 설정

In [ ]:
# LoRA 설정
peft_config = LoraConfig(
    lora_alpha=32,
    lora_dropout=0.1,
    r=16,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]
)

# 모델 준비
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

## 5. 데이터 포맷팅

In [ ]:
def formatting_func(examples):
    """데이터를 Gemma 형식으로 변환"""
    texts = []
    for instruction, input_text, output in zip(examples['instruction'], examples['input'], examples['output']):
        text = f"""<start_of_turn>user
{instruction}

{input_text}<end_of_turn>
<start_of_turn>model
{output}<end_of_turn>"""
        texts.append(text)
    return texts

# 테스트
print(formatting_func(train_dataset[:1])[0][:500])

In [ ]:
# ⚠️ 이 셀은 건너뛰고 다음 셀(Cell 18)을 사용하세요!
# 메모리 최적화 설정은 Cell 18에 있습니다.

print("⚠️ 이 셀을 실행하지 마세요!")
print("⚠️ 대신 Cell 18 (메모리 최적화 버전)을 실행하세요!")

In [ ]:
output_dir = "./gemma3-classical-lit-finetuned"

# A100 40GB 메모리 최적화 설정
training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=3,
    
    # ⚡ A100 40GB 메모리에 맞춘 배치 크기
    per_device_train_batch_size=1,  # 4→1로 감소 (메모리 부족)
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=16,  # 4→16으로 증가 (effective batch=16 유지)
    
    # 학습률 및 스케줄러
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    warmup_ratio=0.03,
    
    # 로깅 및 평가
    logging_steps=10,
    eval_strategy="steps",
    eval_steps=50,  # 200→50 (더 자주 체크)
    
    # 체크포인트 저장
    save_strategy="steps",
    save_steps=500,
    save_total_limit=1,  # 2→1 (디스크 절약)
    
    # 정밀도 최적화
    bf16=True,
    tf32=True,
    
    # 메모리 최적화 (중요!)
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={"use_reentrant": False},
    max_grad_norm=1.0,
    
    # 데이터 로딩
    dataloader_num_workers=2,  # 4→2 (메모리 절약)
    dataloader_pin_memory=True,
    
    # 기타
    report_to="none",
    load_best_model_at_end=False,
)

from trl import SFTTrainer

# SFTTrainer 
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    formatting_func=formatting_func,
    packing=False,
    max_seq_length=512,  # 1024→512 (메모리 절약!)
    dataset_text_field=None,
    tokenizer=tokenizer,
)

print("✅ Trainer 설정 완료 (A100 40GB 메모리 최적화)")
print(f"   Effective batch size: 16 (per_device=1 × grad_accum=16)")
print(f"   Max sequence length: 512")
print(f"   예상 학습 시간: 40-60분")

In [ ]:
print("🚀 학습 시작...")
trainer.train()
print("✅ 학습 완료!")

## 8. 모델 저장

In [ ]:
# 로컬 저장
trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)
print(f"✅ 모델 저장 완료: {output_dir}")

# Cloud Storage에 업로드
!gsutil -m cp -r {output_dir} gs://knu-team-03-data/classical-literature/models/
print("✅ Cloud Storage 업로드 완료")

## 9. 테스트

In [ ]:
# 추론 테스트
test_input = """<start_of_turn>user
다음 지문을 읽고 질문에 답하세요. 학생의 사고를 유도하며 답변을 작성하세요.

[지문]
춘향전의 한 장면...

[질문]
이몽룡이 신분을 숨긴 이유는 무엇인가요?<end_of_turn>
<start_of_turn>model
"""

inputs = tokenizer(test_input, return_tensors="pt").to(model.device)
outputs = model.generate(
    **inputs,
    max_new_tokens=500,
    temperature=0.7,
    top_p=0.9,
    do_sample=True
)
response = tokenizer.decode(outputs[0], skip_special_tokens=False)
print(response)